<a href="https://colab.research.google.com/github/veereshthotigar/CSEE5590-490-AI-CyberSecurity-/blob/master/Lab2/Same%20architecture%20as%20the%20target%20model%20(CNN)/Membership_Attack_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# required imports
import sys 
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline  

import pickle
import torch
import torchvision 
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
# mount the google drive to download the datasets
from google.colab import drive
drive.mount('/content/drive')
project_path = '/content/drive/My Drive/cybersecurity/Lab-Assignment-2/Part-1'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# check if CUDA available or not
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
# clear the cache
torch.cuda.empty_cache()

True
Tesla K80


In [0]:

#load the dataset
with open(project_path+'/data/shadow.data', 'rb') as filehandle:
    # read the data as binary data stream
    predictionsList = pickle.load(filehandle)

In [5]:
    
total_size = len(predictionsList)
split1 = total_size // 4
split1 = total_size - split1 
split2 = split1*2
indices = list(range(total_size))

# two groups to train the shadow (in and out)
train_idx = indices[:] 
test_idx = indices[split1:] 
print(f'No.of train date {len(train_idx)} and No.of test data {len(test_idx)}')
batch_size = 10 # pick your own

# divide and load shadow train in and out
train_sampler = SubsetRandomSampler(train_idx) # Pytorch function
train_loader = torch.utils.data.DataLoader(predictionsList, batch_size=batch_size, sampler=train_sampler)

test_sampler = SubsetRandomSampler(test_idx) # Pytorch function
test_loader = torch.utils.data.DataLoader(predictionsList, batch_size=batch_size, sampler=test_sampler)   


No.of train date 2500 and No.of test data 625


In [6]:
# create the Attack Model: A NN binary classifier {0, 1}
# the input to this model is the propability distribution vector of size 10
# and the output is either 0 (input was not included in training) or 1
from torch.autograd import Variable
attack_model = nn.Sequential(nn.Linear(10, 20),
                      nn.ReLU(),
                      nn.Linear(20, 26),
                      nn.ReLU(),
                      nn.Linear(26, 16),
                      nn.ReLU(),
                      nn.Linear(16, 8),
                      nn.ReLU(),
                      nn.Linear(8, 2),
                      nn.LogSoftmax(dim=1))
attack_model = attack_model.cuda()
attack_criterion = nn.CrossEntropyLoss()
attack_optimizer = optim.SGD(attack_model.parameters(), lr=0.003)

epochs = 5
for e in range(epochs):
    running_loss = 0
    for outputs, labels in train_loader:
        # sending tensors to GPU
        outputs = outputs.cuda().float()
        labels = labels.cuda().long()
        attack_optimizer.zero_grad()
        pred = torch.exp(attack_model(outputs))
        #print(pred.data, labels)
        loss = attack_criterion(pred, labels)
        loss.backward()
        attack_optimizer.step()
        
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(predictionsList)}")
        
torch.save(attack_model.state_dict(), project_path+'/attack_checkpoint.pth')
print('Finished Training the Attack model')

Training loss: 0.06951470029354095
Training loss: 0.06946231746673584
Training loss: 0.06942369985580445
Training loss: 0.06939538474082947
Training loss: 0.0693745816707611
Finished Training the Attack model


In [0]:
#load the dataset
with open(project_path+'/data/target.data', 'rb') as filehandle:
    # read the data as binary data stream
    validation = pickle.load(filehandle)

In [0]:
batch_size = 1 # pick your own

validation_sampler = SubsetRandomSampler(indices[:2400]) # randomly picking 5000 data items
validation_loader = torch.utils.data.DataLoader(validation, batch_size=batch_size, sampler=validation_sampler)

In [26]:
#input the output of the target model to your attack network 
# you already know the target_in and target_out samples, so use that info to evaluate the attack model
correct = 0
incorrect = 0
total = 0
tp = 0
tn = 0
fp = 0
fn = 0
with torch.no_grad():
    for outputs, labels in validation_loader:
        # sending tensors to GPU
        outputs = outputs.cuda().float()
        labels = labels.cuda().long()
        pred = torch.exp(attack_model(outputs))
        predicted = torch.argmax(pred.data)
        #print('\n',pred.data, predicted)
        total += labels.size(0)
        correct += (predicted.item() == labels.item())
        tp += ((predicted.item() == labels.item()) and (predicted.item() == 1))
        tn += ((predicted.item() == labels.item()) and (predicted.item() == 0))
        fp += ((predicted.item() != labels.item()) and (predicted.item() == 1))
        fn += ((predicted.item() != labels.item()) and (predicted.item() == 0))
        #print(predicted.item(),labels.item(),correct)
incorrect = total- correct
print(f'TP : {tp}, TN : {tn}, FP : {fp}, FN : {fn}')
pre = tp/(tp+fp)
rec = tp/(tp+fn)
print(f'Precision {pre*100}')
print(f'Recall {rec*100}')
print(f'F1 Score {2*((pre*rec)/(pre+rec))*100}')


TP : 1250, TN : 0, FP : 1150, FN : 0
Precision 52.083333333333336
Recall 100.0
F1 Score 68.4931506849315


Great! At this point, you must have created a succesfful attack model that can detect whether a datapoint was used in training a target mode or not. 
* A successful attack model is one with a precision/recall higher than 85% -- you are using same architecture and are aware of the data classes

 
 Can you suggest any defense mechanism? If yes, Apply them to your solution and re-evaluate your attack model. How did your defense mecanism affect the accuracy of the target model? How did it affect the recall and precision of the Attack model?